# Geo 2

In [1]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db = client.geo2

## Exercise 1

In [12]:
def find_my_dong(lon: float, lat: float) -> dict:
    return db.hangjeongdong.find_one({'geometry': {'$geoIntersects': {'$geometry': {'type': 'Point', 'coordinates': [lon, lat]}}}})

mydong = find_my_dong(126.954885, 37.482571)
print(mydong['properties']['adm_nm'])

서울특별시 관악구 행운동


## Exercise 2

In [15]:
def get_count_of_bob_in_dong(dong: dict) -> int:
    return db.bob_license.count_documents({'loc': {'$geoWithin': {'$geometry': dong['geometry']}}, 'trdstatenm': '영업/정상'})

cnt = get_count_of_bob_in_dong(mydong)
print(cnt)

177


## Exercise 3

In [98]:
def find_near_dongs(lon: float, lat: float, r: float) -> list:  # list of documents
    db.hangjeongdong.create_index([('geometry', '2dsphere')])
    res = db.hangjeongdong.find({'geometry': {'$nearSphere': {'$geometry': {'type': 'Point', 'coordinates': [lon, lat]}, '$maxDistance': r}}})
    return list(res)

dongs = find_near_dongs(126.954885, 37.482571, 2000)
for dong in dongs:
    cnt = get_count_of_bob_in_dong(dong)
    print(dong['properties']['adm_nm'], cnt)

서울특별시 관악구 행운동 177
서울특별시 관악구 중앙동 73
서울특별시 관악구 낙성대동 604
서울특별시 관악구 청룡동 726
서울특별시 관악구 성현동 26
서울특별시 관악구 청림동 18
서울특별시 동작구 사당5동 43
서울특별시 관악구 은천동 82
서울특별시 관악구 인헌동 182
서울특별시 동작구 상도1동 289
서울특별시 동작구 사당4동 145
서울특별시 동작구 사당3동 33
서울특별시 관악구 대학동 481
서울특별시 관악구 서림동 100
서울특별시 동작구 상도4동 203
서울특별시 관악구 보라매동 25
서울특별시 관악구 서원동 298
서울특별시 동작구 사당2동 174
서울특별시 동작구 사당1동 408
서울특별시 동작구 상도3동 119
서울특별시 관악구 남현동 426
서울특별시 동작구 흑석동 347
서울특별시 관악구 신림동 413
서울특별시 동작구 상도2동 142


## Exercise 4

### Using MultiPolygon

In [100]:
def get_newest_10_bobs(dongs: list):
    multi_poly = {
        'type': 'MultiPolygon',
        'coordinates': []
    }

    for dong in dongs:
        multi_poly['coordinates'].append(dong['geometry']['coordinates'])

    res = db.bob_license.find({'loc': {'$geoWithin': {'$geometry': multi_poly}}}).sort([('apvpermymd', -1)]).limit(10)
    for bob in res:
        print(bob['bplcnm'], bob['apvpermymd'])


get_newest_10_bobs(dongs)

더벤티 신림녹두점 20220310
훈수 삼겹 20220310
마라온 20220302
우리푸드(냠냠숯불두마리치킨) 20220302
티에스 20220302
도스타코스(dostacos) 20220302
복덕밥 20220302
청기와타운 서울대점 20220228
밥찌김밥 20220225
전설낙곱새 관악점 20220225


### Using $or

In [102]:
def get_newest_10_bobs(dongs: list):
    or_list = []

    for dong in dongs:
        or_list.append({'loc': {'$geoWithin': {'$geometry': dong['geometry']}}})

    res = db.bob_license.find({'$or': or_list}).sort([('apvpermymd', -1)]).limit(10)
    for bob in res:
        print(bob['bplcnm'], bob['apvpermymd'])


get_newest_10_bobs(dongs)

더벤티 신림녹두점 20220310
훈수 삼겹 20220310
마라온 20220302
우리푸드(냠냠숯불두마리치킨) 20220302
티에스 20220302
도스타코스(dostacos) 20220302
복덕밥 20220302
청기와타운 서울대점 20220228
밥찌김밥 20220225
전설낙곱새 관악점 20220225
